In [ ]:
# Import libraries

# reload des .py
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from keras_cv.layers import RandomGaussianBlur
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

import numpy as np
import sys
from random import randint, seed
import pathlib
import matplotlib.pyplot as plt
import os
from PIL import Image

from tqdm import tqdm


import dotenv
dotenv.load_dotenv('/tf/science-de-la-donnee/Pipeline_Final/.env')

# ---------------| PATHS |--------------- #
# CHANGER LES PATHS DANS LE FICHIER .env !!!!!!!!!!!!!!
data_dir = os.environ.get('DATA_PATH')
script_path = os.environ.get('SCRIPT_PATH')
weight_path = os.environ.get('WEIGHT_PATH')
models_path = os.environ.get('MODELS_PATH')

output_folder_classification = os.environ.get('OUTPUT_PATH_CLASSIFICATION')
output_folder_autoencoder = os.environ.get('OUTPUT_PATH_AUTOENCODER')

sys.path.insert(0, script_path)
sys.path.insert(1, models_path)

# scripts
import checkImage
import createTrainingData
import displayImage
import plotPrediction
import plotResults
import processImage
import plotImage

# models
import resnet50
import test2


In [ ]:

# ---------------| VARIABLES |--------------- #

# Vérfication des images
check_images = False

model_choosen = 'resnet50'

# Taille de l'image (resacaling)
image_h, image_w = 448, 448
image_size = (image_h, image_w)
# Taille des batchs
batch_size_classification = 1

# Minimum de pixels pour que l'image soit considérée comme une image
min_h = 25
min_w = 25

# Nombre d'epochs
epochs = 15


IMG_SIZE_AUTOENCODER          = 400             # taille coté final d'une image en pixel (ici 224x224)
NB_EPOCHS_AUTOENCODER = 3               # nombre epoch alogithme debruiter
BATCH_SIZE_AUTOENCODER        = 8            # taille batch de traitement

In [ ]:
# # Pré-tri des images (celle qui sont corompus sont écartées)
if check_images:
    checkImage.checkImage(data_dir, min_h, min_w)

image_dir = data_dir

# Utilisez ImageDataGenerator pour charger les images
datagen = ImageDataGenerator()  # Normalisation des pixels

image_generator = datagen.flow_from_directory(
    image_dir,
    target_size=image_size,
    batch_size=batch_size_classification,
    shuffle=False  # Assurez-vous que les images sont dans le bon ordre
)

def load_model(model_choosen):
    match(model_choosen):
        case 'resnet50':
            model = resnet50.build((image_h, image_w), 2)
    return model

model = load_model(model_choosen)

# model = tf.keras.models.load_model(models_path+'/resnet50.keras')
model.load_weights(weight_path+"/classification/cp-0002.ckpt")

# Effectuez des prédictions sur les images
predictions = model.predict(image_generator)

image_dir = data_dir+"/dataset"

# Créer un dictionnaire pour associer les noms de fichiers aux prédictions
image_predictions = {}

# Obtenir la liste des noms de fichiers d'images triés
image_files = sorted(os.listdir(image_dir))

for i in range(len(image_files)):
    # Récupérer le nom de fichier
    filename = image_files[i]

    # Récupérer la prédiction pour cette image
    prediction = predictions[i][0]  # Extrait la valeur scalaire

    # Stocker le nom de fichier et la prédiction dans le dictionnaire
    image_predictions[filename] = prediction

photos, not_photos = plotImage.plot_images_with_highlight(predictions, image_generator,output_folder_classification, label_value=1)

# Copier coller les valeurs prédictes dans le dossier output
import shutil
import os
# Check if folder of photo exists, if not create it
if not os.path.exists(output_folder_classification+"/dataset/Photo/dataset"):
    os.makedirs(output_folder_classification+"/dataset/Photo/dataset")
else:
    # remove all files in folder
    for file in os.listdir(output_folder_classification+"/dataset/Photo/dataset"):
        os.remove(output_folder_classification+"/dataset/Photo/dataset/"+file)

for photo in photos:
    shutil.copy(data_dir+photo, output_folder_classification+"/dataset/Photo/"+photo)

# Check if folder of photo exists, if not create it
if not os.path.exists(output_folder_classification+"/dataset/NotPhoto"):
    os.makedirs(output_folder_classification+"/dataset/NotPhoto")
else:
    # remove all files in folder
    for file in os.listdir(output_folder_classification+"/dataset/NotPhoto"):
        os.remove(output_folder_classification+"/dataset/NotPhoto/"+file)

for photo in not_photos:
    shutil.copy(data_dir+photo, output_folder_classification+"/dataset/NotPhoto/"+photo.replace("dataset/", "")) 

In [ ]:
# Denoising

# ---------------| VARIABLES |--------------- #
data_dir_autotoencoder = output_folder_classification
output_dir_autotoencoder = os.environ.get('OUTPUT_PATH_AUTOENCODER')

MODEL_CHOSEN = "autoencoder_no_pooling"


def load_model(model_choosen):
    match(model_choosen):
        # case 'autoencoder_pooling':
        #     model = homemade.build(IMG_SIZE)
        case 'autoencoder_no_pooling':
            model = test2.build(IMG_SIZE_AUTOENCODER)
        # case 'resnet':
        #     model = resnet.build(IMG_SIZE)
    return model
autoencoder = load_model(MODEL_CHOSEN)

autoencoder.load_weights(weight_path + '/autoencoder/weights_400_2.h5')

datagen = ImageDataGenerator(rescale=1.0/255.0)

image_generator_autoencoder = datagen.flow_from_directory(
    data_dir_autotoencoder+"/dataset/Photo",
    target_size=(IMG_SIZE_AUTOENCODER,IMG_SIZE_AUTOENCODER),
    batch_size=BATCH_SIZE_AUTOENCODER,
    shuffle=False  # Assurez-vous que les images sont dans le bon ordre
)

original_image_paths = image_generator_autoencoder.filenames

# Check if folder of photo exists, if not create it
if not os.path.exists(output_folder_autoencoder+"/denoised_dataset"):
    os.makedirs(output_folder_autoencoder+"/denoised_dataset")
else:
    # remove all files in folder
    for file in os.listdir(output_folder_autoencoder+"/denoised_dataset"):
        os.remove(output_folder_autoencoder+"/denoised_dataset/"+file)

import imageio

for img_index in range(0,len(image_generator_autoencoder)):
    if img_index < len(predictions):
        original_batch = image_generator_autoencoder[img_index][0]
        denoised_batch = autoencoder.predict(original_batch)

    # Parcourez les images dans le batch
    for batch_index in range(len(original_batch)):
        if batch_index < len(original_batch):
            original = original_batch[batch_index]
            denoised = denoised_batch[batch_index]

            # Récupérez le nom de l'image d'origine
            original_filename = os.path.basename(original_image_paths[img_index * BATCH_SIZE_AUTOENCODER + batch_index])

            denoised_image_path = os.path.join(output_folder_autoencoder, "denoised_dataset", "denoised_" + original_filename)

            # Convertissez l'image "denoised" au format 8 bits par canal (0-255)
            denoised = (denoised * 220.0).astype(np.uint8)

            # Enregistrez l'image "denoised" sous forme de fichier
            imageio.imsave(denoised_image_path, denoised)

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog

def classification(DATA_PATH):
    
    def parcourir_dossier():
        dossier_selectionne = filedialog.askdirectory()
        if dossier_selectionne:
            fichier_ou_dossier_var.set(dossier_selectionne)

    def confirmer():
        # Vous pouvez ajouter ici la logique pour traiter les données et effectuer la classification
        # Pour l'instant, affichons simplement les valeurs sélectionnées
        result_text = f"Chemin du fichier/dossier : {fichier_ou_dossier_var.get()}\n"
        result_text += f"Hauteur de l'image : {input_image_size.get()}\n"
        result_text += f"Largeur de l'image : {input_image_width.get()}\n"
        result_text += f"Hauteur de l'image minimale : {input_min_image_size.get()}\n"
        result_text += f"Largeur de l'image minimale : {input_min_image_width.get()}\n"
        result_text += f"Modèle de classification : {input_model.get()}\n"
        result_text += f"Taille du batch : {input_batch_size.get()}\n"
        result_text += f"Nombre d'epochs : {input_epochs.get()}\n"
        result_text += f"Check d'image : {check_image_var.get()}\n"
        result_text += f"Type de classification : {classification_var.get()}\n"
        label_result.config(text=result_text)

    fenetre = tk.Tk()
    fenetre.title("Interface Tkinter")
    fenetre.geometry("500x700")

    titre = tk.Label(fenetre, text="Classification", font=("Helvetica", 16))
    titre.pack(pady=10)

    separator = ttk.Separator(fenetre, orient='horizontal')
    separator.pack(fill='x')

    classification_frame = tk.Frame(fenetre)
    classification_frame.pack(pady=5)
    label_classification = tk.Label(classification_frame, text="Classification")
    label_classification.pack(side=tk.LEFT)

    fichier_ou_dossier_var = tk.StringVar()
    fichier_ou_dossier_var.set(DATA_PATH)  # Texte de placeholder
    fichier_ou_dossier_entry = tk.Entry(fenetre, textvariable=fichier_ou_dossier_var, width=40)
    fichier_ou_dossier_entry.pack(pady=5)
    bouton_parcourir = tk.Button(fenetre, text="Parcourir", command=parcourir_dossier)
    bouton_parcourir.pack()

    label_image_size = tk.Label(fenetre, text="Hauteur de l'image:")
    label_image_size.pack()
    input_image_size = tk.Entry(fenetre)
    input_image_size.pack(pady=5)

    label_image_width = tk.Label(fenetre, text="Largeur de l'image:")
    label_image_width.pack()
    input_image_width = tk.Entry(fenetre)
    input_image_width.pack(pady=5)

    label_min_image_size = tk.Label(fenetre, text="Hauteur de l'image minimale:")
    label_min_image_size.pack()
    input_min_image_size = tk.Entry(fenetre)
    input_min_image_size.pack(pady=5)

    label_min_image_width = tk.Label(fenetre, text="Largeur de l'image minimale:")
    label_min_image_width.pack()
    input_min_image_width = tk.Entry(fenetre)
    input_min_image_width.pack(pady=5)

    label_model = tk.Label(fenetre, text="Modèle de classification:")
    label_model.pack()
    input_model = tk.Entry(fenetre)
    input_model.pack(pady=5)

    label_batch_size = tk.Label(fenetre, text="Taille du batch:")
    label_batch_size.pack()
    input_batch_size = tk.Entry(fenetre)
    input_batch_size.pack(pady=5)

    label_epochs = tk.Label(fenetre, text="Nombre d'epochs:")
    label_epochs.pack()
    input_epochs = tk.Entry(fenetre)
    input_epochs.pack(pady=5)

    check_image_var = tk.IntVar()
    check_image_button = tk.Checkbutton(fenetre, text="Check d'image", variable=check_image_var)
    check_image_button.pack(pady=5)

    classification_var = tk.StringVar()
    classification_var.set("Binaire")
    label_classification_type = tk.Label(fenetre, text="Type de classification:")
    label_classification_type.pack()
    classification_binaire_button = tk.Radiobutton(fenetre, text="Binaire", variable=classification_var, value="Binaire")
    classification_multi_classes_button = tk.Radiobutton(fenetre, text="Multi-classes", variable=classification_var, value="Multi-classes")
    classification_binaire_button.pack()
    classification_multi_classes_button.pack()

    bouton = tk.Button(fenetre, text="Confirmer", command=confirmer)
    bouton.pack(pady=10)

    label_result = tk.Label(fenetre, text="")
    label_result.pack(pady=10)

    fenetre.mainloop()

classification("DATA_PATH")